In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.0674],
        [-0.0513]], grad_fn=<AddmmBackward0>)

In [2]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.2448, -0.2119, -0.2276, -0.2813,  0.1519, -0.2975,  0.1186, -0.0110]])), ('bias', tensor([0.2101]))])


In [4]:
print(net[0].state_dict())

OrderedDict([('weight', tensor([[-0.4584,  0.3636,  0.4937,  0.2158],
        [ 0.1428,  0.3798,  0.4377,  0.0234],
        [ 0.2600, -0.4323, -0.0745,  0.1908],
        [ 0.4880,  0.3435, -0.1359,  0.2488],
        [-0.1401, -0.1035,  0.0879, -0.3015],
        [-0.0860,  0.4611,  0.0063, -0.2110],
        [ 0.2838, -0.3492,  0.3963, -0.2684],
        [-0.0605,  0.1418,  0.3988,  0.2228]])), ('bias', tensor([-0.0528,  0.1972,  0.1339,  0.3806,  0.2576,  0.1261,  0.3595,  0.4704]))])


In [5]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2101], requires_grad=True)
tensor([0.2101])


In [6]:
net[2].weight.grad == None

True

In [10]:
net[0].weight.grad == None

True

In [11]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [12]:
net.state_dict()['2.bias'].data

tensor([0.2101])

In [13]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.3391],
        [0.3391]], grad_fn=<AddmmBackward0>)

In [14]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [15]:
rgnet[0][1][0].bias.data

tensor([-0.2529,  0.4443,  0.3630, -0.1329,  0.4157, -0.0828, -0.1425, -0.0283])

In [16]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0229, -0.0108,  0.0132,  0.0257]), tensor(0.))

In [17]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [18]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.6505, -0.5582,  0.4622, -0.4916])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [23]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight, net[2].weight

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


(Parameter containing:
 tensor([[-7.3412, -7.5044,  6.0596, -0.0000],
         [-6.3630, -0.0000, -0.0000, -8.0450],
         [-0.0000, -0.0000, -0.0000, -0.0000],
         [ 0.0000, -0.0000, -5.7253, -0.0000],
         [-0.0000, -0.0000,  0.0000,  0.0000],
         [ 0.0000, -9.8968, -6.8046, -5.0234],
         [ 5.9109, -0.0000, -0.0000, -0.0000],
         [-0.0000, -0.0000,  7.4960,  5.0087]], requires_grad=True),
 Parameter containing:
 tensor([[-9.9147,  6.8573, -6.0610,  0.0000,  0.0000, -0.0000,  5.8281, -0.0000]],
        requires_grad=True))

In [24]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -6.5044,  7.0596,  1.0000])

In [25]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
